## 형태소분석_명사추출

- 형태소 분석을 먼저하고
- 거기서 띄어쓰기없이 쭉 이어져 있는 부분들에 대하여 인위적으로 띄어쓰기 넣어주기

In [1]:
with open('data/cleaned_playlist_pje.txt') as f:
    playlist = f.read().splitlines()
    
with open('data/preanal.txt') as f:
    data = f.read().splitlines()

In [2]:
len(playlist)

115071

In [3]:
playlist

['여행같은 음악',
 '요즘 너 말야',
 '편하게  잔잔하게 들을 수 있는 곡  ',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래  ',
 '2017 Pop Trend',
 '짝사랑  고백  사랑  이별  슬픔   감성을 자극하는곡들 ',
 '멍청이   내맘도 몰라 ',
 'DANCING IN THE MOON LIGHT  01',
 ' 록 메탈  Written by 이일우',
 '걸그룹 땐쓰쏭 ',
 '노래로 의지를 불태우자   1일1다짐  st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합 알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면  너도 내 생각 할까  ',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때 새벽감성',
 'ᴡʜɪᴛᴇ   ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ',
 '추억의 명화를 만나는 시간  영화 OST 모음 ',
 '  기분좋은 여름날  ',
 '지친 하루 끝  힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때   국외 ',
 '기분좋은 햇살같은 숨겨진 명곡들  ',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지  미국 대표 클래식',
 '겨울이 기다리며 미리 들어요   ',
 '아침이 상쾌해지는 빅밴드의 스윙스윙  ',
 '여행가는 길 이런음악과 함께 고고씽   ',
 '믿고듣는 하우스 일렉트로니카 POP  2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자 ',
 '발라드로 감성을 발라드림 슬픔주의 ',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '8 90년대 우리나라 라디오에서 흘러나오던 명곡들 ',
 '영화  Jazz를 만나다 294',

In [4]:
for i in range(0,len(playlist)):
    p = re.compile('[^ㄱ-힗]\s[^ㄱ-힗]')
    if p.match(playlist[i]) !=None:
        print(i,p.match(playlist[i]),'yes')
#     if len(playlist[i])==0:
#         print('yes')

NameError: name 're' is not defined

### 명사추출

In [5]:
import re
import itertools
nouns = []

# 모든 NNP/NNG/NP추출
for i in range(0,len(data)):
   tmp = list([item for item in data[i].replace('\t'," ").split(" ") if re.search('(NNP$)|(NNG$)|(NP$)', item) and (len(item)>=6)])
   if len(tmp)!=0:
       nouns.append(tmp)

#nouns unlist 진행
nouns = list(itertools.chain(*nouns))
nouns_final = list(set([re.sub("/NNG|/NNP|/NP","",item) for item in nouns])) #최종 preanal에 있는 unique 명사

## Khaiii ㄱㄱ

In [6]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

**이거를 병렬처리**

In [ ]:
morphs = []
tmp_list = []

for i in range(0,len(playlist)):
    if playlist[i].find(" ")==-1: #띄어쓰기 없는 것만 수동띄어쓰기
        for j in range(0,len(nouns_final)):
            playlist[i] = re.sub("(\w*)("+nouns_final[j]+")(\w+)|(\w+)("+nouns_final[j]+")(\s+)|(\w+)("+nouns_final[j]+")(\w+)",r"\1 \2 \3",playlist[i])
        print(playlist[i])
    if playlist[i].isspace():
        continue
    for word in api.analyze(playlist[i]):
        for morph in word.morphs:
            # 실질형태소 가져오기
            if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ']:
                tmp_list.append(morph.lex)
    morphs.append(tmp_list)
    tmp_list = []
    print(i)

### 병렬처리

### Pool

In [7]:
from multiprocessing import Pool
import time

def do_work(i):
    try:
        tmp_list = []

        if playlist[i].find(" ")==-1: #띄어쓰기 없는 것만 수동띄어쓰기
            for j in range(0,len(nouns_final)):
                playlist[i] = re.sub("(\w*)("+nouns_final[j]+")(\w+)|(\w+)("+nouns_final[j]+")(\s+)|(\w+)("+nouns_final[j]+")(\w+)",r"\1 \2 \3",playlist[i])
            print(playlist[i])
        for word in api.analyze(playlist[i]):
            for morph in word.morphs:
                # 실질형태소 가져오기
                if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ','SN','SL']:
                    tmp_list.append(morph.lex)
        print(i)
        return tmp_list
    except Exception as ex: # 에러 종류
        tmp_list
        print("error occur ",i, ex)


if __name__ == '__main__':
    start_time = time.time()
    with Pool(processes=4) as p:
        result = p.map(do_work, range(0,len(playlist)))
    print("--- %s seconds ---" % (time.time()-start_time))

21576
7192014384
21577


7193114385


7194214386


7195314387


7196414388


7197514389


614390

714391

8
14392
914393

10
11
12
13
14
 겨울 이네요피겨스케이팅
 수정 이앨범
21578
7198
14394
21579
7199
14395
21580
7200걍 게임 할때듣는음악
14396


2158172011514397



7202error occur 14398 16

215827203
 14399


72041440021583


72051440121584


720614402

720714403

720814404

720914405

721014406

721114407

721214408

721314409

721414410

7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
축가듀엣
차 장님 과나17

2158518

2158619

2158720

2158821

2221589

2321590

24
2159125

2621592

2721593

21594
21595
21596
21597
21598
21599
21600
21601
21602
21603
21604
21605
21606
21607
한경일
14411
error occur  14412 
14413
14414
14415
14416
14417
14418
14419
14420
14421
14422
14423
14424
14425
14426
호림뮤직
7243
7244
7245
7246
7247
7248
7249
즐겨듣는소래
28
트로트129

3021608

3121609

3221610

3321611

3421612

3521613

21614
21615
21616
21617 자주 듣는

In [30]:
result

[['여행', '같', '음악'],
 ['요즘', '너', '말'],
 ['편하', '잔잔', '듣', '있', '곡'],
 ['크리스마스', '분위기', '흠뻑', '취하', '때'],
 ['추억', '노래'],
 [],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
 ['멍청', '나', '맘', '모르'],
 [],
 ['록', '메', '탈', '이일우'],
 ['걸그룹', '때', '쓰쏭'],
 ['노래', '의지', '불태우', '짐', '프로', '필뮤직'],
 ['지치', '너', '위로', '제목', '편지', '준비'],
 ['트렌디', '그루브', '힙합', '알앤비'],
 [],
 ['걍', '게임', '하', '때', '듣', '음악'],
 ['나', '알', '노래'],
 ['축가듀엣'],
 ['이렇', '비', '내리', '날', '너', '나', '생각', '하'],
 ['올림픽', '스밍', '목록'],
 ['조용히', '맥주', '한잔', '하', '때', '새벽', '감성'],
 ['ᴡ', 'ᴛɪ', 'ᴊ', 'ᴄ'],
 ['추억', '명화', '만나', '시간', '영화', '모음'],
 ['기분', '좋', '여름날'],
 ['지치', '하루', '끝', '힐', '링', '필요', '당신', '추천', '인디곡'],
 ['새벽', '찬바람', '침대위', '감', '쌀', '때', '국외'],
 ['기분', '좋', '햇살', '같', '숨기', '명곡'],
 ['사랑', '남녀', '듀엣'],
 ['즐기', '듣', '소래'],
 ['닥힙추'],
 ['거', '슈윈', '스타', '워', '즈', '미국', '대표', '클래식'],
 ['겨울', '기다리', '미리', '들'],
 ['아침', '상쾌', '빅', '밴', '드', '스윙스윙'],
 ['여행가', '길', '이런', '음', '악', '함께', '고고씽'],
 ['믿', '듣', '하우스', '일렉트로니카'],

In [15]:
len(result)

115071

In [18]:
16035/60/60

4.454166666666667

In [23]:
result[1]

['요즘', '너', '말']

### None으로 들어간 데이터들 빈칸으로 수정

In [44]:
for i in range(0,len(result)):
    if result[i] == None:
        print(i, 'yes')

219 yes
518 yes
799 yes
1697 yes
1960 yes
2047 yes
2055 yes
2916 yes
3030 yes
3212 yes
3429 yes
3508 yes
3639 yes
3970 yes
5119 yes
5229 yes
5358 yes
5545 yes
6569 yes
7122 yes
7186 yes
7809 yes
8393 yes
8603 yes
9902 yes
10070 yes
10386 yes
10953 yes
11205 yes
11566 yes
11716 yes
11881 yes
11909 yes
12068 yes
12399 yes
12646 yes
12679 yes
13256 yes
13388 yes
14026 yes
14293 yes
14412 yes
14680 yes
14794 yes
14932 yes
15802 yes
16012 yes
16723 yes
16856 yes
17128 yes
17686 yes
17740 yes
17813 yes
18214 yes
18467 yes
18862 yes
19237 yes
19385 yes
19493 yes
19710 yes
20006 yes
20252 yes
21077 yes
21444 yes
21462 yes
21582 yes
21766 yes
22181 yes
22364 yes
22535 yes
22585 yes
22610 yes
23079 yes
23597 yes
24034 yes
24255 yes
24323 yes
24340 yes
24804 yes
24891 yes
24962 yes
25594 yes
26049 yes
26705 yes
27058 yes
27667 yes
28033 yes
28062 yes
28175 yes
28373 yes
28661 yes
29142 yes
29158 yes
29591 yes
29782 yes
30255 yes
30880 yes
31015 yes
31082 yes
31109 yes
31397 yes
31747 yes
32510 ye

In [45]:
for i in range(0,len(result)):
    if result[i] == None:
        result[i] = []
        print(i, result[i])

219 []
518 []
799 []
1697 []
1960 []
2047 []
2055 []
2916 []
3030 []
3212 []
3429 []
3508 []
3639 []
3970 []
5119 []
5229 []
5358 []
5545 []
6569 []
7122 []
7186 []
7809 []
8393 []
8603 []
9902 []
10070 []
10386 []
10953 []
11205 []
11566 []
11716 []
11881 []
11909 []
12068 []
12399 []
12646 []
12679 []
13256 []
13388 []
14026 []
14293 []
14412 []
14680 []
14794 []
14932 []
15802 []
16012 []
16723 []
16856 []
17128 []
17686 []
17740 []
17813 []
18214 []
18467 []
18862 []
19237 []
19385 []
19493 []
19710 []
20006 []
20252 []
21077 []
21444 []
21462 []
21582 []
21766 []
22181 []
22364 []
22535 []
22585 []
22610 []
23079 []
23597 []
24034 []
24255 []
24323 []
24340 []
24804 []
24891 []
24962 []
25594 []
26049 []
26705 []
27058 []
27667 []
28033 []
28062 []
28175 []
28373 []
28661 []
29142 []
29158 []
29591 []
29782 []
30255 []
30880 []
31015 []
31082 []
31109 []
31397 []
31747 []
32510 []
33061 []
33151 []
33465 []
33626 []
34087 []
34173 []
34192 []
34381 []
34820 []
35396 []
35500 []
35

### 수정됐는지 확인

In [46]:
for i in range(0,len(result)):
    if result[i] == None:
        print(i, 'yes')

### 결과 저장

In [47]:
result

[['여행', '같', '음악'],
 ['요즘', '너', '말'],
 ['편하', '잔잔', '듣', '있', '곡'],
 ['크리스마스', '분위기', '흠뻑', '취하', '때'],
 ['추억', '노래'],
 [],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
 ['멍청', '나', '맘', '모르'],
 [],
 ['록', '메', '탈', '이일우'],
 ['걸그룹', '때', '쓰쏭'],
 ['노래', '의지', '불태우', '짐', '프로', '필뮤직'],
 ['지치', '너', '위로', '제목', '편지', '준비'],
 ['트렌디', '그루브', '힙합', '알앤비'],
 [],
 ['걍', '게임', '하', '때', '듣', '음악'],
 ['나', '알', '노래'],
 ['축가듀엣'],
 ['이렇', '비', '내리', '날', '너', '나', '생각', '하'],
 ['올림픽', '스밍', '목록'],
 ['조용히', '맥주', '한잔', '하', '때', '새벽', '감성'],
 ['ᴡ', 'ᴛɪ', 'ᴊ', 'ᴄ'],
 ['추억', '명화', '만나', '시간', '영화', '모음'],
 ['기분', '좋', '여름날'],
 ['지치', '하루', '끝', '힐', '링', '필요', '당신', '추천', '인디곡'],
 ['새벽', '찬바람', '침대위', '감', '쌀', '때', '국외'],
 ['기분', '좋', '햇살', '같', '숨기', '명곡'],
 ['사랑', '남녀', '듀엣'],
 ['즐기', '듣', '소래'],
 ['닥힙추'],
 ['거', '슈윈', '스타', '워', '즈', '미국', '대표', '클래식'],
 ['겨울', '기다리', '미리', '들'],
 ['아침', '상쾌', '빅', '밴', '드', '스윙스윙'],
 ['여행가', '길', '이런', '음', '악', '함께', '고고씽'],
 ['믿', '듣', '하우스', '일렉트로니카'],

In [88]:
i=0     
text = open('data/cleaned_playlist_title.txt','w')
while i < len(result):
    for j in range(0,len(result[i])):
        text.write(result[i][j])
        if j < len(result[i])-1:
            text.write(',')
        else: continue
    text.write('\n')
    i += 1
text.close()